In [ ]:
from flask import Flask, render_template_string, request, session, redirect, url_for
import random
import smtplib
from email.mime.text import MIMEText
from datetime import datetime

app = Flask(__name__)
app.secret_key = 'your_secret_key'

# HTML Templates
signup_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sign Up</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            margin: 0;
            padding: 0;
            font-size: 16px; /* Increase the base font size */
            }
        
        .signup-container {
            margin: 90px auto;
            width: 500px;
            height: auto; /* Adjust height as needed, or use padding for height */
            background-color: white;
            padding: 40px; /* Increase padding for more height */
            border-radius: 20px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            }
        h2 {
            text-align: center;
            color: #333;
        }
        .input-container {
            position: relative;
            margin-bottom: 20px;
        }
        .input-container input {
            width: 90%;
            padding: 12px; /* Increase padding for better spacing */
            padding-left: 40px; /* Space for the icon */
            border: 1px solid #ccc;
            border-radius: 5px;
            font-size: 16px; /* Increase input font size */
        }
        .input-container i {
            position: absolute;
            left: 10px; /* Adjust icon position */
            top: 50%; /* Center the icon vertically */
            transform: translateY(-50%); /* Adjust for vertical centering */
            font-size: 18px;
            color: #007BFF;
            pointer-events: none; /* Prevent interaction with the icon */
        }
        button {
            width: 100%;
            padding: 12px; /* Increase padding for button */
            background-color: #007BFF;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 16px; /* Ensure button font size is consistent */
        }
        button:hover {
            background-color: #0056b3;
        }
        img.logo {
            position: absolute;
            top: 10px;
            left: 20px;
            width: 200px;
        }
    </style>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">
</head>
<body>
    <img src="https://cuvette.tech/assets/images/programPage/cuvetteLogo.png" alt="Logo" class="logo">
    <div class="signup-container">
        <h2>Sign Up</h2>
        <form action="/submit_signup" method="POST">
            <div class="input-container">
                <i class="fas fa-user"></i>
                <input type="text" id="name" name="name" placeholder="Name" required>
            </div>
            <div class="input-container">
                <i class="fas fa-envelope"></i>
                <input type="email" id="email" name="email" placeholder="Email" required>
            </div>
            <div class="input-container">
                <i class="fas fa-building"></i>
                <input type="text" id="company-name" name="company-name" placeholder="Company Name" required>
            </div>
            <div class="input-container">
                <i class="fas fa-envelope-open"></i>
                <input type="email" id="company-email" name="company-email" placeholder="Company Email" required>
            </div>
            <div class="input-container">
                <i class="fas fa-users"></i>
                <input type="text" id="employee-size" name="employee-size" placeholder="Employee Size" required>
            </div>
            <button type="submit">Sign Up</button>
        </form>
    </div>
</body>
</html>

"""

otp_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>OTP Verification</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            margin: 0;
            padding: 0;
        }
        .otp-container {
            margin: 80px auto;
            width: 500px;
            background-color: white;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        h2 {
            text-align: center;
            color: #333;
        }
        label {
            display: block;
            margin: 10px 0 5px;
        }
        input {
            width: 100%;
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        button {
            width: 100%;
            padding: 10px;
            background-color: #007BFF;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 16px;
        }
        button:hover {
            background-color: #0056b3;
        }
        .contact {
            position: absolute;
            top: 10px;
            right: 20px;
        }
        .contact a {
            color: #007BFF;
            text-decoration: none;
            font-size: 30px; 
        }
        img.logo {
            position: absolute;
            top: 10px;
            left: 20px;
            width: 200px;
        }
    </style>
</head>
<body>
    <img src="https://cuvette.tech/assets/images/programPage/cuvetteLogo.png" alt="Logo" class="logo">
    <div class="contact">
        <a href="/contact">Contact</a>
    </div>
    <div class="otp-container">
        <h2>OTP Verification</h2>
        <form action="/verify" method="POST">
            <label for="email-otp">Enter OTP:</label>
            <input type="text" id="email-otp" name="email-otp" required>
            <button type="submit">Proceed</button>
        </form>
    </div>
</body>
</html>
"""

third_page_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Job Posting</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            margin: 0;
            padding: 0;
        }
        .job-container {
            margin: 80px auto;
            width: 500px;
            background-color: white;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        }
        h2 {
            text-align: center;
            color: #333;
        }
        label {
            display: block;
            margin: 10px 0 5px;
        }
        input, textarea {
            width: 100%;
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        button {
            width: 100%;
            padding: 10px;
            background-color: #007BFF;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 16px;
        }
        button:hover {
            background-color: #0056b3;
        }
        .nav-links {
            position: absolute;
            top: 10px;
            right: 20px;
        }
        .nav-links a {
            color: #007BFF;
            text-decoration: none;
            font-size: 30px;
            margin-left: 20px;
        }
        img.logo {
            position: absolute;
            top: 10px;
            left: 20px;
            width: 200px;
        }
    </style>
</head>
<body>
    <img src="https://cuvette.tech/assets/images/programPage/cuvetteLogo.png" alt="Logo" class="logo">
    <div class="nav-links">
        <a href="/contact">Contact</a>
        <a href="/account-details">Your Account</a>
    </div>
    <div class="job-container">
        <h2>Post a Job</h2>
        <form action="/submit_job" method="POST">
            <label for="job-title">Job Title:</label>
            <input type="text" id="job-title" name="job-title" required>

            <label for="description">Description:</label>
            <textarea id="description" name="description" rows="4" required></textarea>

            <label for="experience-level">Experience Level:</label>
            <input type="text" id="experience-level" name="experience-level" required>

            <label for="candidate-details">Candidate Details (Emails):</label>
            <textarea id="candidate-details" name="candidate-details" rows="4" required></textarea>

            <label for="end-date">End Date:</label>
            <input type="date" id="end-date" name="end-date" required>

            <button type="submit">Send</button>
        </form>
    </div>
</body>
</html>
"""

# Routes and functions
@app.route('/')
def index():
    return render_template_string(signup_html)

@app.route('/submit_signup', methods=['POST'])
def submit_signup():
    # Store user details in session
    session['name'] = request.form['name']  # Added name field
    session['email'] = request.form['email']
    session['company_name'] = request.form['company-name']
    session['company_email'] = request.form['company-email']
    session['employee_size'] = request.form['employee-size']
    
    # Generate OTP and store in session
    otp = random.randint(100000, 999999)
    session['otp'] = otp

    # Send OTP via email
    user_email = session['email']
    send_email(user_email, "Your OTP Code", f"Your OTP is {otp}")

    return redirect(url_for('otp'))

@app.route('/otp')
def otp():
    return render_template_string(otp_html)

@app.route('/verify', methods=['POST'])
def verify():
    entered_otp = request.form['email-otp']
    
    if entered_otp == str(session.get('otp')):
        return redirect(url_for('job_posting'))
    else:
        return "Invalid OTP, please try again."

@app.route('/job_posting')
def job_posting():
    return render_template_string(third_page_html)

@app.route('/submit_job', methods=['POST'])
def submit_job():
    job_title = request.form['job-title']
    description = request.form['description']
    experience_level = request.form['experience-level']
    candidate_emails = request.form['candidate-details'].split(',')
    end_date = request.form['end-date']

    # Send job posting details to candidate emails
    for email in candidate_emails:
        send_email(email, f"Job Opportunity: {job_title}", f"Description: {description}\nExperience Level: {experience_level}\nEnd Date: {end_date}")
    
    return "Job information has been sent to all candidates."

@app.route('/contact')
def contact():
    return render_template_string("""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Contact Support</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #f4f4f4;
                margin: 0;
                padding: 0;
            }
            .contact-container {
                margin: 80px auto;
                width: 500px;
                background-color: white;
                padding: 30px;
                border-radius: 10px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            }
            h2 {
                text-align: center;
                color: #333;
            }
            label {
                display: block;
                margin: 10px 0 5px;
            }
            input, textarea {
                width: 100%;
                padding: 10px;
                margin-bottom: 10px;
                border: 1px solid #ccc;
                border-radius: 5px;
            }
            button {
                width: 100%;
                padding: 10px;
                background-color: #007BFF;
                color: white;
                border: none;
                border-radius: 5px;
                cursor: pointer;
                font-size: 16px;
            }
            button:hover {
                background-color: #0056b3;
            }
            img.logo {
                position: absolute;
                top: 10px;
                left: 20px;
                width: 200px;
        }
        </style>
    </head>
    <body>
        <img src="https://cuvette.tech/assets/images/programPage/cuvetteLogo.png" alt="Logo" class="logo">
        <div class="contact-container">
            <h2>Contact Support</h2>
            <form action="/submit_contact" method="POST">
                <label for="name">Name:</label>
                <input type="text" id="name" name="name" required>

                <label for="email">Email:</label>
                <input type="email" id="email" name="email" required>

                <label for="issue">Issue:</label>
                <textarea id="issue" name="issue" rows="4" required></textarea>

                <button type="submit">Submit</button>
            </form>
        </div>
    </body>
    </html>
    """)

@app.route('/submit_contact', methods=['POST'])
def submit_contact():
    name = request.form['name']
    email = request.form['email']
    issue = request.form['issue']
    
    # Send email to support team
    support_email = 'balakabharanireddy@gmail.com'
    send_email(support_email, "Support Issue", f"Name: {name}\nEmail: {email}\nIssue: {issue}")
    
    # Send confirmation to user
    send_email(email, "Support Confirmation", "Your issue has been sent to the support team. Please wait, we will get back to you soon!")

    return redirect(url_for('contact'))

@app.route('/account-details')
def account_details():
    return f"""
    <style>
        body {{
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            margin: 0;
            padding: 0;
        }}
        .account-container {{
            max-width: 600px;
            margin: 100px auto;
            padding: 20px;
            background-color: white;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            text-align: center;
        }}
        h2 {{
            color: #333;
        }}
        .detail {{
            display: flex;
            align-items: center;
            margin: 15px 0;
            font-size: 18px;
        }}
        .detail i {{
            margin-right: 10px;
            color: #007BFF;
        }}
    </style>
    
    <div class="account-container">
        <h2>Your Account Details</h2>
        <div class="detail">
            <i class="fas fa-user"></i>  <!-- Name Icon -->
            <p>Name: {session.get('name')}</p>
        </div>
        <div class="detail">
            <i class="fas fa-envelope"></i>  <!-- Email Icon -->
            <p>Email: {session.get('email')}</p>
        </div>
        <div class="detail">
            <i class="fas fa-building"></i>  <!-- Company Name Icon -->
            <p>Company Name: {session.get('company_name')}</p>
        </div>
        <div class="detail">
            <i class="fas fa-envelope-square"></i>  <!-- Company Email Icon -->
            <p>Company Email: {session.get('company_email')}</p>
        </div>
        <div class="detail">
            <i class="fas fa-users"></i>  <!-- Employee Size Icon -->
            <p>Employee Size: {session.get('employee_size')}</p>
        </div>
    </div>
    """


# Helper function to send email
def send_email(to_email, subject, body):
    from_email = 'balakabharanireddy@gmail.com'
    from_password = 'xavc dxqm tqdn minw'
    
    # Create the email message
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email

    # Send the email
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(from_email, from_password)
        server.sendmail(from_email, to_email, msg.as_string())

# Run the app
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)  # Disable the use of reloader to avoid watchdog import


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Oct/2024 14:41:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 14:41:18] "POST /submit_signup HTTP/1.1" 302 -
127.0.0.1 - - [19/Oct/2024 14:41:18] "GET /otp HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 14:41:26] "POST /verify HTTP/1.1" 302 -
127.0.0.1 - - [19/Oct/2024 14:41:26] "GET /job_posting HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 14:41:28] "GET /account-details HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 14:41:38] "GET /account-details HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 14:42:13] "GET /account-details HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2024 14:42:17] "GET /contact HTTP/1.1" 200 -
